In [476]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [ ]:
prism_addr = 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'
yluna_addr = 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'
pluna_addr = 'terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2'

In [482]:
class DataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.refract = 'c5ac5e60-7da0-429f-98e8-19ccdd77d835'
        self.refract_cluna = '5b0257c3-e93b-49d4-93f6-e370bc3b3f50'
        self.yluna_lping = '879d0260-93a9-4576-a1bd-2cc3dc26bf13'
        self.pluna_lping = '7f5c7008-648a-4944-9fe9-e6c37e2e7bb8'
        self.ystaking = '3ff0fc49-5a0d-4cdf-a8ab-33f8ea7e755f'
        self.ystaking_farm = '05d91866-0193-4231-b2ca-1774fbd5742a'
        self.swaps = '1bfd8019-89a1-470d-8868-60d71e57d1d0'
        self.router = '69b149fb-81ba-4860-aac1-b17f0d6d7688'
        self.collector = '2ab62a07-3882-48e6-bdc6-d9e592aee2d8'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
        self.dates_to_mark = pd.DataFrame([
            ['2022-02-13', '2022-02-13',2000000,'Prism Forge'], 
            ['2022-03-06', '2022-03-06',2400000,'Prism Farm']], 
            columns=['text_date','date','height','text']
        )
        
    def load_from_url(self):
        self.ystaking_farm_df = self.claim(self.ystaking_farm)
        self.yluna_lping_df = self.claim(self.yluna_lping)
        self.pluna_lping_df = self.claim(self.pluna_lping)
        self.refract_df = self.claim(self.refract)
        self.refract_cluna_df = self.claim(self.refract_cluna)
        self.ystaking_df = self.claim(self.ystaking)
        self.swaps_df = self.claim(self.swaps)
        self.router_df = self.claim(self.router)
        self.collector_df = self.claim(self.collector)
    
    def write_to_csv(self):
        self.ystaking_farm_df.to_csv(f'{self.path_to_data}/ystaking_farm.csv')
        self.yluna_lping_df.to_csv(f'{self.path_to_data}/yluna_lping.csv')
        self.pluna_lping_df.to_csv(f'{self.path_to_data}/pluna_lping.csv')
        self.refract_df.to_csv(f'{self.path_to_data}/refract.csv')
        self.refract_cluna_df.to_csv(f'{self.path_to_data}/refract_cluna.csv')
        self.ystaking_df.to_csv(f'{self.path_to_data}/ystaking.csv')
        self.swaps_df.to_csv(f'{self.path_to_data}/swaps.csv')
        self.router_df.to_csv(f'{self.path_to_data}/router.csv')
        self.collector_df.to_csv(f'{self.path_to_data}/collector.csv')
        
    def load_from_csv(self):
        self.ystaking_farm_df = pd.read_csv(f'{self.path_to_data}/ystaking_farm.csv', index_col=0)
        self.yluna_lping_df = pd.read_csv(f'{self.path_to_data}/yluna_lping.csv', index_col=0)
        self.refract_df = pd.read_csv(f'{self.path_to_data}/refract.csv', index_col=0)
        self.refract_cluna_df = pd.read_csv(f'{self.path_to_data}/refract_cluna.csv', index_col=0)
        self.ystaking_df = pd.read_csv(f'{self.path_to_data}/ystaking.csv', index_col=0)
        self.swaps_df = pd.read_csv(f'{self.path_to_data}/swaps.csv', index_col=0)
        self.router_df = pd.read_csv(f'{self.path_to_data}/router.csv', index_col=0)
        self.collector_df = pd.read_csv(f'{self.path_to_data}/collector.csv', index_col=0)
        
    def polish_yluna_lping(self):
        self.yluna_lping_df['action'] = self.yluna_lping_df.apply(lambda row: row['0_action'] if row['0_action'] == 'provide_liquidity' else row['1_action'],axis=1)
        self.yluna_lping_df.block_timestamp=self.yluna_lping_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.yluna_lping_df.block_timestamp=self.yluna_lping_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        provide_ = self.yluna_lping_df[self.lping_df.action=='provide_liquidity']
        withdraw_ = self.yluna_lping_df[self.lping_df.action=='withdraw_liquidity']
        #
        provide_['prism_amount'] = provide_.apply(lambda row: row['2_amount'] if row['1_contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw' else row['3_amount'],axis=1)
        provide_['yluna_amount'] = provide_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz' else row['2_amount'],axis=1)
        provide_['sender'] = provide_['from_']
        provide_['hr'] = provide_.block_timestamp.str[:-5] + '00:00.000'
        provide_['day'] = provide_.block_timestamp.str[:-9]
        provide_ = provide_[['block_timestamp','sender','tx_id','action','prism_amount','yluna_amount','hr','day']]
        provide_['amount_signed'] = provide_.yluna_amount
        provide_['type'] = 'provide_lp'
        self.provide_ = provide_
        #
        withdraw_['prism_amount'] = withdraw_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw' else row['2_amount'],axis=1)
        withdraw_['yluna_amount'] = withdraw_.apply(lambda row: row['2_amount'] if row['3_contract_address'] == 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz' else row['1_amount'],axis=1)
        withdraw_['sender'] = withdraw_['from_']
        withdraw_['hr'] = withdraw_.block_timestamp.str[:-5] + '00:00.000'
        withdraw_['day'] = withdraw_.block_timestamp.str[:-9]
        withdraw_ = withdraw_[['block_timestamp','sender','tx_id','action','prism_amount','yluna_amount','hr','day']]
        withdraw_['amount_signed'] = -withdraw_.yluna_amount
        withdraw_['type'] = 'withdraw_lp'
        self.withdraw_ = withdraw_

        
    def polish_refracting_cluna(self):
        self.refract_cluna_df['user'] = self.refract_cluna_df['from_']
        self.refract_cluna_df_pol = self.refract_cluna_df[['block_timestamp','tx_id','user','0_action','0_amount']]
        self.refract_cluna_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']
        self.refract_cluna_df_pol['asset_given'] = 'cLUNA'
        self.refract_cluna_df_pol['asset_received'] = 'yLUNA'
        self.refract_cluna_df_pol['operation'] = 'refraction'
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.refract_cluna_df_pol['hr'] = self.refract_cluna_df_pol.block_timestamp.str[:-5] + '00:00.000'
        self.refract_cluna_df_pol['day'] = self.refract_cluna_df_pol.block_timestamp.str[:-9]
        self.refract_cluna_df_pol['amount_signed'] = self.refract_cluna_df_pol.amount
        
    def polish_refracting_luna(self):
        self.refract_df['user'] = self.refract_df.apply(lambda row: row.from_ if row['0_action']=='bond_split' else row.to_, axis=1)
        self.refract_df_pol = self.refract_df[['block_timestamp','tx_id','user','0_action','0_amount']]
        self.refract_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']
        self.refract_df_pol['asset_given'] = self.refract_df_pol.apply(lambda row: 'LUNA' if row['action']=='bond_split' else 'yLUNA',axis=1)
        self.refract_df_pol['asset_received'] = self.refract_df_pol.apply(lambda row: 'yLUNA' if row['action']=='bond_split' else 'LUNA',axis=1)
        self.refract_df_pol['operation'] = 'refraction'
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.refract_df_pol['hr'] = self.refract_df_pol.block_timestamp.str[:-5] + '00:00.000'
        self.refract_df_pol['day'] = self.refract_df_pol.block_timestamp.str[:-9]
        self.refract_df_pol['amount_signed'] = self.refract_df_pol.apply(lambda row: -row.amount 
                                                        if row.action=='burn_from' else row.amount,axis=1)
        
    def polish_refracting(self):
        self.polish_refracting_cluna()
        self.polish_refracting_luna()
        self.all_refreact = self.refract_df_pol.append(self.refract_cluna_df_pol)
        daily_delta_rf = self.all_refreact.groupby('day').amount_signed.sum().reset_index()
        daily_delta_rf = daily_delta_rf.sort_values(by='day')
        daily_delta_rf['cumsum'] = daily_delta_rf.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_rf.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_rf['Type'] = 'yLuna circulating'
        self.daily_delta_rf = daily_delta_rf
        
    def polish_ystaking(self):
        self.ystaking_df['action'] = self.ystaking_df.apply(lambda row: row['0_action'] if row['0_action']=='unbond' else row['1_action'],axis=1)
        self.ystaking_df['amount'] = self.ystaking_df.apply(lambda row: row['0_amount'] if row['0_action']=='unbond' else row['1_amount'],axis=1)
        self.ystaking_df['user'] = self.ystaking_df.apply(lambda row: row['to_'] if row.action=='unbond' else row['from_'],axis=1)
        self.ystaking_df['asset_given'] = self.ystaking_df.apply(lambda row: None if row.action=='unbond' else 'yLUNA',axis=1)
        self.ystaking_df['asset_received'] = self.ystaking_df.apply(lambda row: 'yLUNA' if row.action=='unbond' else None,axis=1)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.ystaking_df['day'] = self.ystaking_df.hr.apply(str).str[:-13]
        self.ystaking_df = self.ystaking_df[['block_timestamp','tx_id','hr','day','action','amount','user','asset_given','asset_received']]
        self.ystaking_df['amount_signed'] = self.ystaking_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)
        
    def polish_ystaking_farm(self):
        self.ystaking_farm_df['action'] = self.ystaking_farm_df.apply(lambda row: row['3_action'] if row['3_action']=='bond' else row['1_action'], axis=1)
        self.ystaking_farm_df['amount'] = self.ystaking_farm_df.apply(lambda row: float(row['0_amount']) if row['action']=='bond' else float(row['0_amount']), axis=1)
        self.ystaking_farm_df = self.ystaking_farm_df[['block_timestamp','tx_id','sender','action','amount']]
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.ystaking_farm_df['hr'] = self.ystaking_farm_df.block_timestamp.str[:-5] + '00:00.000'
        self.ystaking_farm_df['day'] = self.ystaking_farm_df.block_timestamp.str[:-9]
        self.ystaking_farm_df['amount_signed'] = self.ystaking_farm_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)
        
        
    def polish_simple_swaps(self):
        swaps_df = self.swaps_df[self.swaps_df.ask_asset != '']
        swaps_df = swaps_df[swaps_df.sender.notna()]
        swaps_df_pol = swaps_df.rename(columns={'sender':'user','ask_asset':'asset_received','offer_asset':'asset_given'})
        swaps_df_pol = swaps_df_pol[['block_timestamp','tx_id','price','user','asset_received','return_amount','asset_given','offer_amount']]
        swaps_df_pol['operation'] = 'swap'
        self.swaps_df_pol = swaps_df_pol
    
    def polish_router(self):
        router_df = self.router_df.rename(columns={'sender':'user'})
        router_df_1 = router_df[['block_timestamp','tx_id','0_ask_asset','0_offer_amount','0_offer_asset','0_price','0_return_amount','user']]
        router_df_1 = router_df_1.rename(columns={'0_ask_asset':'asset_received','0_offer_amount':'offer_amount',
                           '0_price':'price','0_return_amount':'return_amount','0_offer_asset':'asset_given'})
        router_df_2 = router_df[['block_timestamp','tx_id','1_ask_asset','1_offer_amount','1_offer_asset','1_price','1_return_amount','user']]
        router_df_2 = router_df_2.rename(columns={'1_ask_asset':'asset_received','1_offer_amount':'offer_amount',
                           '1_price':'price','1_return_amount':'return_amount','1_offer_asset':'asset_given'})
        router_df_pol = router_df_1.append(router_df_2)
        router_df_pol['operation'] = 'swap'
        self.router_df_pol = router_df_pol
    
    def polish_swaps(self):
        self.polish_simple_swaps()
        self.polish_router()
        self.swaps_df_all = self.router_df_pol.append(self.swaps_df_pol[self.router_df_pol.columns])
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.swaps_df_all['hr'] = self.swaps_df_all.block_timestamp.str[:-5] + '00:00.000'
        self.swaps_df_all['day'] = self.swaps_df_all.block_timestamp.str[:-9]
        #
        yluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='yLUNA')|(self.swaps_df_all.asset_received=='yLUNA')]
        yluna_swaps['amount_signed'] = yluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'yLUNA' else -row.return_amount,axis=1)
        yluna_swaps['type'] = 'swap'
        self.yluna_swaps = yluna_swaps
    
    def get_amount_yluna(self,row):
        #Swapping Prism for yLuna
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}offer_asset'] == 'cw20:terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'):
                return row[f'{prefix}offer_amount']
        #Asking for yLuna from the pool
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}ask_asset'] == 'cw20:terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'):
                return -row[f'{prefix}return_amount']
        #Swapping PRISM for yLuna
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] == 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'):
                if(row[f'{prefix}from'] == 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'):
                    if(row[f'{prefix}action'] == 'swap'):
                        return row[f'{i}_amount']
        #Sending yLuna to the pool
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] == 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'):
                if(row[f'{prefix}from'] == 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'):
                    if(row[f'{prefix}action'] == 'send'):
                        return row[f'{prefix}amount']
    
    def polish_collector_and_others(self):
        lp_txs = self.withdraw_[['tx_id']].append(self.provide_[['tx_id']])\
                            .append(self.yluna_swaps[['tx_id']])
        self.collector_df.block_timestamp=self.collector_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.collector_df.block_timestamp=self.collector_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.collector_df['hr'] = self.collector_df.block_timestamp.str[:-5] + '00:00.000'
        self.collector_df['day'] = self.collector_df.block_timestamp.str[:-9]
        self.collector_df = self.collector_df[self.collector_df.tx_id.isin(set(dp.collector_df.tx_id.unique()).difference(lp_txs.tx_id.unique()))]
        self.collector_df['amount_signed'] = self.collector_df.apply(self.get_amount_yluna,axis=1)/1000000
        self.collector_df['type'] = 'collector_and_other'
        
    def polish(self):
        self.polish_yluna_lping()
        self.polish_refracting()
        self.polish_ystaking()
        self.polish_ystaking_farm()
        self.polish_swaps()
        self.polish_collector_and_others()
    
    def lp_delta(self):
        self.all_lps = self.withdraw_[['day','amount_signed','tx_id','type','block_timestamp']]\
                            .append(self.provide_[['day','amount_signed','tx_id','type','block_timestamp']])\
                            .append(self.yluna_swaps[['day','amount_signed','tx_id','type','block_timestamp']])\
                            .append(self.collector_df[['day','amount_signed','tx_id','type','block_timestamp']])
        daily_delta_lp = self.all_lps.groupby('day').amount_signed.sum().reset_index()
        daily_delta_lp = daily_delta_lp.sort_values(by='day')
        daily_delta_lp['cumsum'] = daily_delta_lp.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_lp.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_lp['Type'] = 'yLuna LP'
        self.daily_delta_lp = daily_delta_lp
        
    def stk_farm_delta(self):
        daily_delta_stk_farm = self.ystaking_farm_df.groupby('day').amount_signed.sum().reset_index()
        daily_delta_stk_farm = daily_delta_stk_farm.sort_values(by='day')
        daily_delta_stk_farm['cumsum'] = daily_delta_stk_farm.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_stk_farm.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_stk_farm['Type'] = 'yLuna Farm staked'
        self.daily_delta_stk_farm = daily_delta_stk_farm
        
    def stk_delta(self):
        daily_delta_stk = self.ystaking_df.groupby('day').amount_signed.sum().reset_index()
        daily_delta_stk = daily_delta_stk.sort_values(by='day')
        daily_delta_stk['cumsum'] = daily_delta_stk.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_stk.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_stk['Type'] = 'yLuna staked'
        self.daily_delta_stk = daily_delta_stk
    
    def unused_yluna(self):
        self.y_used = self.all_deltas[self.all_deltas.Type.isin(['yLuna LP','yLuna staked','yLuna Farm staked'])]
        y_tot = self.all_deltas[self.all_deltas.Type.isin(['yLuna circulating'])]
        y_unused = self.y_used.groupby('Time').Amount.sum().rename('Amount_used').reset_index().merge(y_tot, on='Time')
        y_unused['Amount_delta'] = y_unused['Amount'] - y_unused['Amount_used']
        y_unused['Amount_delta'] = y_unused['Amount_delta'].apply(lambda x: x if x>0 else 0)
        y_unused = y_unused[['Time','Amount signed','Amount_delta','Type']]
        y_unused['Type'] = 'yLuna unused'
        y_unused.columns = ['Time','Amount signed','Amount','Type']
        self.y_unused = y_unused
        
    def daily_delta(self):
        self.lp_delta()
        self.stk_farm_delta()
        self.stk_delta()
        self.all_deltas = self.daily_delta_rf.append(self.daily_delta_stk)\
                                    .append(self.daily_delta_lp)\
                                    .append(self.daily_delta_stk_farm)
        self.unused_yluna()
        
        
    def fill_date_gaps(self, dff, extra_dates=[]):
        dd = dff.Time.unique()
        dd = [*dd,*extra_dates]
        unique_dates = pd.Series(dd).rename('Time').reset_index().drop(columns='index')
        for t in dff.Type.unique():
            df = dff[dff.Type==t]
            last_date = df.Time.max()
            last_value = df[df.Time==last_date].Amount.values[0]
            df = df.merge(unique_dates, on='Time', how='right')
            df = df[df.Type.isna()]
            df['Type'] = t
            df['Amount'] = df.apply(lambda row: last_value if row.Time>last_date else 0,axis=1)
            dff = dff.append(df.fillna(0))
        return dff


In [483]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [484]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [485]:
class ChartProvider:
    def __init__(self):
        pass
    
    def get_yluna_time_area_chart(self, df, scale_, min_date=None, max_date=None, top_padding=0):
        max_date = df['Time'].max()
        df['Amount (millions)'] = round(df['Amount']/1000000,2).apply(str)+'M'
        chart = alt.Chart(df).mark_area().encode(
            x=alt.X('Time:T',scale=alt.Scale(domain=(min_date,max_date))),
            y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,df['Amount'].max()+top_padding))),
            color=alt.Color('Type:N', 
                        scale=scale_,
                        legend=alt.Legend(
                                    orient='none',
                                    padding=5,
                                    legendY=0,
                                    direction='vertical')),
            tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d'),'Type:N','Amount (millions):N']
        )
        return chart

In [596]:
class SwapsDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.swaps = '1bfd8019-89a1-470d-8868-60d71e57d1d0'
        self.router = '69b149fb-81ba-4860-aac1-b17f0d6d7688'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
            
    def load_from_url(self):
        self.swaps_df = self.claim(self.swaps)
        self.router_df = self.claim(self.router)
        
        
    def parse_simple_swaps(self):
        swaps_df = self.swaps_df[self.swaps_df.ask_asset != '']
        swaps_df = swaps_df[swaps_df.sender.notna()]
        swaps_df_pol = swaps_df.rename(columns={'sender':'user','ask_asset':'asset_received','offer_asset':'asset_given'})
        swaps_df_pol = swaps_df_pol[['block_timestamp','tx_id','price','user','asset_received','return_amount','asset_given','offer_amount']]
        swaps_df_pol['operation'] = 'swap'
        self.swaps_df_pol = swaps_df_pol
        
    def parse_router(self):
        router_df = self.router_df.rename(columns={'sender':'user'})
        router_df_1 = router_df[['block_timestamp','tx_id','0_ask_asset','0_offer_amount','0_offer_asset','0_price','0_return_amount','user']]
        router_df_1 = router_df_1.rename(columns={'0_ask_asset':'asset_received','0_offer_amount':'offer_amount',
                           '0_price':'price','0_return_amount':'return_amount','0_offer_asset':'asset_given'})
        router_df_2 = router_df[['block_timestamp','tx_id','1_ask_asset','1_offer_amount','1_offer_asset','1_price','1_return_amount','user']]
        router_df_2 = router_df_2.rename(columns={'1_ask_asset':'asset_received','1_offer_amount':'offer_amount',
                           '1_price':'price','1_return_amount':'return_amount','1_offer_asset':'asset_given'})
        router_df_pol = router_df_1.append(router_df_2)
        router_df_pol['operation'] = 'swap'
        self.router_df_pol = router_df_pol
    
    def parse(self):
        self.parse_simple_swaps()
        self.parse_router()
        self.swaps_df_all = self.router_df_pol.append(self.swaps_df_pol[self.router_df_pol.columns])
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.swaps_df_all['hr'] = self.swaps_df_all.block_timestamp.str[:-5] + '00:00.000'
        self.swaps_df_all['day'] = self.swaps_df_all.block_timestamp.str[:-9]
        #
        yluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='yLUNA')|(self.swaps_df_all.asset_received=='yLUNA')]
        yluna_swaps['amount_signed'] = yluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'yLUNA' else -row.return_amount,axis=1)
        yluna_swaps['type'] = 'swap'
        self.yluna_swaps = yluna_swaps
        #
        pluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='pLUNA')|(self.swaps_df_all.asset_received=='pLUNA')]
        pluna_swaps['amount_signed'] = pluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'pLUNA' else -row.return_amount,axis=1)
        pluna_swaps['type'] = 'swap'
        self.pluna_swaps = pluna_swaps
        

In [597]:
swaps_dp = SwapsDataProvider(claim,get_url,'../data')
swaps_dp.load_from_url()
swaps_dp.parse()

In [588]:
class LPDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.lp_provide_withdraw = '7f5c7008-648a-4944-9fe9-e6c37e2e7bb8'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
            
    def load_from_url(self):
        self.lp_provide_withdraw_df = self.claim(self.lp_provide_withdraw)
        
    def get_action(self, row):
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(f'{prefix}action' in row):
                if(row[f'{prefix}action'] in ['provide_liquidity','withdraw_liquidity']):
                    return row[f'{prefix}action']
                
    def get_n_action(self, row):
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(f'{prefix}action' in row):
                if(row[f'{prefix}action'] in ['provide_liquidity','withdraw_liquidity']):
                    return f'{prefix}'
                
    def correct_parsing(self, row):
        if (row.prefix == '1_' and row.f_action == 'withdraw_liquidity') or \
            (row.prefix == '0_' and row.f_action == 'provide_liquidity'):
            return False
        else:
            return True
                
    def parse(self):
        df = self.lp_provide_withdraw_df
        df['f_action'] = df.apply(self.get_action,axis=1)
        df['prefix'] = df.apply(self.get_n_action,axis=1)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x) 
        df['f_contract_address'] = df.apply(lambda row: row[f'{row.prefix}contract_address'], axis=1)
        df['asset'] = df.apply(lambda row: 'yLuna' if row.f_contract_address=='terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce' \
                                    else 'pLuna', axis=1)
        assert df.apply(self.correct_parsing , axis=1).sum() == 0
        provide_ = df[df.f_action=='provide_liquidity']
        withdraw_ = df[df.f_action=='withdraw_liquidity']
        #
        provide_['prism_amount'] = provide_.apply(lambda row: row['2_amount'] if row['1_contract_address'] == prism_addr else row['3_amount'],axis=1)
        provide_['asset_amount'] = provide_.apply(lambda row: row['1_amount'] if row['2_contract_address'] in [yluna_addr,pluna_addr] else row['2_amount'],axis=1)
        provide_['sender'] = provide_['from_']
        provide_['hr'] = provide_.block_timestamp.str[:-5] + '00:00.000'
        provide_['day'] = provide_.block_timestamp.str[:-9]
        provide_ = provide_[['block_timestamp','sender','tx_id','f_action','prism_amount','asset','asset_amount','hr','day']]
        provide_['amount_signed'] = provide_.asset_amount
        provide_['type'] = 'provide_lp'
        self.provide_ = provide_
        #
        withdraw_['prism_amount'] = withdraw_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == prism_addr else row['2_amount'],axis=1)
        withdraw_['asset_amount'] = withdraw_.apply(lambda row: row['2_amount'] if row['3_contract_address'] in [yluna_addr,pluna_addr] else row['1_amount'],axis=1)
        withdraw_['sender'] = withdraw_['from_']
        withdraw_['hr'] = withdraw_.block_timestamp.str[:-5] + '00:00.000'
        withdraw_['day'] = withdraw_.block_timestamp.str[:-9]
        withdraw_ = withdraw_[['block_timestamp','sender','tx_id','f_action','prism_amount','asset','asset_amount','hr','day']]
        withdraw_['amount_signed'] = -withdraw_.asset_amount
        withdraw_['type'] = 'withdraw_lp'
        self.withdraw_ = withdraw_

In [589]:
lp_dp = LPDataProvider(claim,get_url,'../data')
lp_dp.load_from_url()
lp_dp.parse()

In [451]:
dp = DataProvider(claim,get_url,'../data')
dp.load_from_csv()
dp.polish()
dp.daily_delta()

### Check LP txs

In [452]:
#Check the txs missing for the pool (get complete list from ET)
contract = 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'
txs = pd.read_csv(f'../data/txs/{contract}.csv', index_col=0)
txs.timestamp = pd.to_datetime(txs.timestamp)

In [453]:
txs = txs[txs.timestamp.apply(str) < dp.all_lps.block_timestamp.max()]
txs.head()

,hash,timestamp
0,975E19763B9EAAC9BF2D0E3FD142E18714D77D72DB2C78B7176E5E3581ADFAC4,2022-03-10 16:44:17+00:00
1,51DA951E49B713818C95A619D9D2D41DEE6FB09951873E2C8A2842EC8B1338B6,2022-03-10 16:43:12+00:00
2,3A0EE768FCEA4F767E661C1CA9AF6D8B6887DB5699AAB451AE6DD0993937A995,2022-03-10 16:39:59+00:00
3,5DD336F9AEEABFE640141FB170BD9C572B74F577B15434AC34832A7029221412,2022-03-10 16:36:48+00:00
4,A21C0ABECC3C07F8B59CE9921F5116B5F90ED0BBEC698AABD10144989C05AA10,2022-03-10 16:32:03+00:00


In [454]:
missing_txs = set(txs.hash.values).difference(set(dp.all_lps.tx_id.values))
#missing_txs
len(missing_txs)

76

In [455]:
dp.all_lps[dp.all_lps.amount_signed.isna()]

,day,amount_signed,tx_id,type,block_timestamp
1590,2022-02-14,NaN,F1371E3F465062633DF33F66CAA3CD8BF430FB010C38EFDE7F99ECBB4D3F6FD3,collector_and_other,2022-02-14 06:56:57


In [456]:
extra_txs = set(dp.all_lps.tx_id.values).difference(set(txs.hash.values))

In [457]:
len(extra_txs)

472

In [458]:
dp.all_lps[dp.all_lps.tx_id.isin(extra_txs)].groupby('type').amount_signed.sum()

type
collector_and_other   -1091.231453
provide_lp                7.354729
swap                   3526.457907
withdraw_lp             -37.602728
Name: amount_signed, dtype: float64

## Refract

In [459]:
dp.refract_cluna_df.block_timestamp

0       2022-02-17 20:26:14.994
1       2022-02-17 12:37:33.023
2       2022-02-17 09:28:56.174
3       2022-02-17 18:46:03.713
4       2022-02-16 08:40:44.246
                 ...           
2545    2022-03-08 22:08:44.078
2546    2022-03-08 22:11:51.643
2547    2022-03-08 22:17:10.249
2548    2022-03-08 22:24:59.365
2549    2022-03-08 23:44:57.710
Name: block_timestamp, Length: 2550, dtype: object

In [460]:
dp.refract_cluna_df_pol.block_timestamp

0       2022-02-17 20:26:14
1       2022-02-17 12:37:33
2       2022-02-17 09:28:56
3       2022-02-17 18:46:03
4       2022-02-16 08:40:44
               ...         
2545    2022-03-08 22:08:44
2546    2022-03-08 22:11:51
2547    2022-03-08 22:17:10
2548    2022-03-08 22:24:59
2549    2022-03-08 23:44:57
Name: block_timestamp, Length: 2550, dtype: object

In [461]:
daily_delta_rf = dp.daily_delta_rf

In [462]:
daily_delta_rf.head()

,Time,Amount signed,Amount,Type
0,2022-02-14,890610.824139,890610.82,yLuna circulating
1,2022-02-15,213620.887294,1104231.71,yLuna circulating
2,2022-02-16,36275.244129,1140506.96,yLuna circulating
3,2022-02-17,390554.870005,1531061.83,yLuna circulating
4,2022-02-18,19638.781506,1550700.61,yLuna circulating


In [463]:
cp = ChartProvider()
domain = ['yLuna circulating']
range_ = ['#f8936d']
cp.get_yluna_time_area_chart(daily_delta_rf, 
               alt.Scale(domain=domain, range=range_),
               min_date = daily_delta_rf.Time.min(),
               max_date = daily_delta_rf.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## yStaking

In [464]:
def get_max_domain_date(df, time_field, n_hours):
    if((pd.Timestamp(df[time_field].max()) - 
                pd.Timestamp(df[time_field].min())).total_seconds()/3600 < n_hours):
        max_date = (pd.Timestamp(df[time_field].min()) + pd.to_timedelta(n_hours, unit='h')).strftime("%Y-%m-%dT%H:%M:%SZ")
    else:
        max_date = df[time_field].max()
    return max_date

In [465]:
daily_delta_stk = dp.daily_delta_stk

In [466]:
domain = ['yLuna staked']
range_ = ['#f8936d']
cp.get_yluna_time_area_chart(daily_delta_stk, 
               alt.Scale(domain=domain, range=range_),
               min_date = daily_delta_stk.Time.min(),
               max_date = daily_delta_stk.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## yLuna LP

In [467]:
(dp.all_lps.groupby(['tx_id','amount_signed']).count()>1).sum()

day                0
type               0
block_timestamp    0
dtype: int64

In [468]:
dp.collector_df.day

0        2022-03-10
4        2022-03-11
5        2022-03-11
10       2022-03-11
11       2022-03-11
            ...    
18484    2022-03-01
18486    2022-03-01
18488    2022-03-01
18491    2022-03-01
18492    2022-03-01
Name: day, Length: 3840, dtype: object

In [469]:
cp.get_yluna_time_area_chart(dp.daily_delta_lp, 
               alt.Scale(scheme='set2'),
               min_date = daily_delta_stk.Time.min(),
               max_date = daily_delta_stk.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [470]:
dp.all_lps.groupby(['day']).amount_signed.sum()

day
2022-02-14    32029.903413
2022-02-15    15533.023953
2022-02-16    -3949.600484
2022-02-17    -6834.842364
2022-02-18    -7036.598141
2022-02-19      337.493477
2022-02-20    -1702.456873
2022-02-21    -1555.101548
2022-02-22     3529.424816
2022-02-23     5290.555899
2022-02-24    -6865.751919
2022-02-25    -2172.205694
2022-02-26    -2031.778980
2022-02-27    -2120.002002
2022-02-28    -6805.338279
2022-03-01     1265.766605
2022-03-02     1194.081579
2022-03-03     1484.890647
2022-03-04      900.399076
2022-03-05     2973.347473
2022-03-06     1880.293529
2022-03-07    -9020.900787
2022-03-08     -254.783570
2022-03-09      238.098743
2022-03-10     -356.045593
2022-03-11     1922.105632
Name: amount_signed, dtype: float64

In [471]:
cp.get_yluna_time_area_chart(dp.daily_delta_stk_farm, 
               alt.Scale(scheme='set2'),
               min_date = dp.daily_delta_stk_farm.Time.min(),
               max_date = dp.daily_delta_stk_farm.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [472]:
cp.get_yluna_time_area_chart(dp.y_unused, 
               alt.Scale(scheme='set2'),
               min_date = dp.y_unused.Time.min(),
               max_date = dp.y_unused.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## All deltas

In [473]:
all_deltas = dp.y_used.append(dp.y_unused)

In [474]:
all_deltas = dp.fill_date_gaps(all_deltas, ['2022-02-12','2022-02-13'])

In [475]:
c1 = cp.get_yluna_time_area_chart(all_deltas, 
               alt.Scale(scheme='set2'),
               min_date = all_deltas.Time.min(),
               max_date = all_deltas.Time.max(),
               top_padding = 1500000
        )

c2 = alt.Chart(dp.dates_to_mark).mark_rule(color='#e45756').encode(
    x=alt.X('date'+':T',axis=alt.Axis(labels=False,title=''))
)

c3 = alt.Chart(dp.dates_to_mark).mark_text(
    color='#e45756',
    angle=0
).encode(
    x=alt.X('text_date'+':T',axis=alt.Axis(labels=False,title='')),
    y='height',
    text='text'
)

(c1 + c2 + c3).properties(width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

## Query

In [231]:
import requests
luna_apr = float(
    requests.get('https://api.terra.dev/chart/staking-return/annualized').json(
    )[-1]['value']) * 100
et_query = requests.get(
    'https://api.extraterrestrial.money/v1/api/prices').json()
yluna_price = float(et_query['prices']['yLUNA']['price'])
luna_price = float(et_query['prices']['LUNA']['price'])
yluna_apr = luna_apr * luna_price / yluna_price

In [232]:
yluna_apr

8.115326633560468

In [233]:
luna_apr

6.740162722440834

In [234]:
luna_price

95.2243989664808

In [235]:
df = pd.DataFrame(requests.get('https://api.terra.dev/chart/staking-return/annualized').json())
df['date'] = pd.to_datetime(df['datetime'], unit='ms')
df = df[df['date'] > '2021-07-05 15:00:00']

In [236]:
df.value = df.value.apply(float).apply(lambda x: x*100)

In [237]:
mars_price_chart = alt.Chart(df).mark_line(point = True).encode(
    x=alt.X('date:T'),
    y=alt.X('value:Q'),
    tooltip=[alt.Tooltip('date:T', format='%Y-%m-%d %H:%M'),'value:Q']
).properties(width=800).properties(height=300).configure_view(strokeOpacity=0).interactive()
mars_price_chart

alt.Chart(...)